# Session 1: LLM Fundamentals & API Usage

**Duration**: 60 minutes

In this session, you'll learn how to work with Large Language Models through APIs, understand tokens, manage costs, and build your first AI chatbot.

## What You'll Build
- Basic chatbot with conversation history
- Token counter and cost calculator
- Streaming chat interface
- **SupportGenie v0.1**: First version of capstone project

---

## Setup: Install Dependencies

In [ ]:
# Install required libraries
!pip install -q openai tiktoken anthropic google-generativeai

print("✅ All dependencies installed!")

In [ ]:
# Configure API keys
import os

try:
    from google.colab import userdata
    os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
    print("✅ API key loaded from Colab Secrets")
except:
    from getpass import getpass
    os.environ['OPENAI_API_KEY'] = getpass('Enter your OpenAI API key: ')
    print("✅ API key set")

## Part 1: Understanding Tokens

In [ ]:
import tiktoken

def count_tokens(text, model="gpt-3.5-turbo"):
    """Count tokens in text for a specific model"""
    encoding = tiktoken.encoding_for_model(model)
    tokens = encoding.encode(text)
    return len(tokens)

# Test with examples
examples = [
    "Hello, world!",
    "The quick brown fox jumps over the lazy dog",
    "OpenAI creates advanced AI systems"
]

print("Token Counting Examples:")
print("=" * 60)
for text in examples:
    tokens = count_tokens(text)
    print(f"Text: {text}")
    print(f"Tokens: {tokens}")
    print(f"Chars/Token: {len(text)/tokens:.2f}")
    print("-" * 60)

### 🎯 Challenge 1.1: Count Your Own Tokens

Count tokens in a paragraph of your choice!

In [ ]:
# Your code here
my_text = """Add your text here..."""

# Count tokens
token_count = count_tokens(my_text)
print(f"Your text has {token_count} tokens")

## Part 2: Your First API Call

In [ ]:
from openai import OpenAI

# Initialize client
client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))

# Make your first API call
print("Making API call...\n")

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is the capital of France?"}
    ],
    max_tokens=50,
    temperature=0.7
)

# Extract and display answer
answer = response.choices[0].message.content
print(f"Answer: {answer}")
print(f"\nTokens used: {response.usage.total_tokens}")
print(f"  - Prompt: {response.usage.prompt_tokens}")
print(f"  - Completion: {response.usage.completion_tokens}")

### 🎯 Challenge 1.2: Ask Your Own Question

In [ ]:
# Modify the question
my_question = "What is machine learning?"  # Change this!

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": my_question}
    ],
    max_tokens=100
)

print(response.choices[0].message.content)

## Part 3: Exploring Temperature

In [ ]:
def test_temperature(prompt, temperatures=[0.0, 0.5, 1.0, 1.5]):
    """Test the same prompt with different temperatures"""
    
    print(f"Prompt: {prompt}")
    print("=" * 70)
    
    for temp in temperatures:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=temp,
            max_tokens=50
        )
        
        print(f"\nTemperature {temp}:")
        print(response.choices[0].message.content)
        print("-" * 70)

# Test it!
test_temperature("Name a color")

**Observation**: Notice how higher temperatures give more varied/creative responses!

## Part 4: Streaming Responses

In [ ]:
def stream_chat(message):
    """Stream a chat response token by token"""
    
    stream = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": message}],
        stream=True,
        max_tokens=200
    )
    
    full_response = ""
    print("Assistant: ", end="", flush=True)
    
    for chunk in stream:
        if chunk.choices[0].delta.content:
            content = chunk.choices[0].delta.content
            print(content, end="", flush=True)
            full_response += content
    
    print()  # New line
    return full_response

# Test streaming
response = stream_chat("Tell me a short story about a robot")

## Part 5: Cost Calculator

In [ ]:
def calculate_cost(input_text, output_text, model="gpt-3.5-turbo"):
    """Calculate cost of an API call"""
    
    # Pricing per 1M tokens (December 2024)
    pricing = {
        "gpt-3.5-turbo": {"input": 0.50, "output": 1.50},
        "gpt-4-turbo": {"input": 10.00, "output": 30.00},
        "gpt-4": {"input": 30.00, "output": 60.00},
    }
    
    # Count tokens
    input_tokens = count_tokens(input_text, model)
    output_tokens = count_tokens(output_text, model)
    
    # Calculate cost
    input_cost = (input_tokens / 1_000_000) * pricing[model]["input"]
    output_cost = (output_tokens / 1_000_000) * pricing[model]["output"]
    total_cost = input_cost + output_cost
    
    return {
        "input_tokens": input_tokens,
        "output_tokens": output_tokens,
        "total_tokens": input_tokens + output_tokens,
        "input_cost": input_cost,
        "output_cost": output_cost,
        "total_cost": total_cost
    }

# Test it
prompt = "Explain machine learning in simple terms"
response_text = "Machine learning is a way for computers to learn from data without being explicitly programmed."

cost_info = calculate_cost(prompt, response_text)

print("Cost Breakdown:")
print("=" * 60)
print(f"Input tokens: {cost_info['input_tokens']}")
print(f"Output tokens: {cost_info['output_tokens']}")
print(f"Total tokens: {cost_info['total_tokens']}")
print(f"\nInput cost: ${cost_info['input_cost']:.6f}")
print(f"Output cost: ${cost_info['output_cost']:.6f}")
print(f"Total cost: ${cost_info['total_cost']:.6f}")

### 🎯 Challenge 1.3: Model Cost Comparison

Calculate costs for the same text using different models.

In [ ]:
prompt = "Write a 100-word essay about AI"
sample_response = "Artificial intelligence represents..." * 10  # Simulate response

models = ["gpt-3.5-turbo", "gpt-4-turbo", "gpt-4"]

print("Model Cost Comparison:")
print("=" * 60)
for model in models:
    cost = calculate_cost(prompt, sample_response, model)
    print(f"{model:20} ${cost['total_cost']:.6f}")

## Part 6: Building a Simple Chatbot

In [ ]:
class SimpleChatbot:
    def __init__(self, api_key, model="gpt-3.5-turbo"):
        self.client = OpenAI(api_key=api_key)
        self.model = model
        self.conversation_history = []
        self.total_tokens = 0
        self.total_cost = 0.0
    
    def set_system_message(self, message):
        """Set the system prompt"""
        self.conversation_history = [
            {"role": "system", "content": message}
        ]
    
    def chat(self, user_message, stream=False):
        """Send a message and get response"""
        
        # Add user message
        self.conversation_history.append({
            "role": "user",
            "content": user_message
        })
        
        if stream:
            return self._stream_response()
        else:
            return self._complete_response()
    
    def _complete_response(self):
        """Get complete response at once"""
        response = self.client.chat.completions.create(
            model=self.model,
            messages=self.conversation_history,
            temperature=0.7,
            max_tokens=500
        )
        
        assistant_message = response.choices[0].message.content
        
        # Add to history
        self.conversation_history.append({
            "role": "assistant",
            "content": assistant_message
        })
        
        # Track usage
        self.total_tokens += response.usage.total_tokens
        
        return assistant_message
    
    def _stream_response(self):
        """Stream response token by token"""
        stream = self.client.chat.completions.create(
            model=self.model,
            messages=self.conversation_history,
            temperature=0.7,
            max_tokens=500,
            stream=True
        )
        
        full_response = ""
        for chunk in stream:
            if chunk.choices[0].delta.content:
                content = chunk.choices[0].delta.content
                print(content, end="", flush=True)
                full_response += content
        
        print()  # New line
        
        # Add to history
        self.conversation_history.append({
            "role": "assistant",
            "content": full_response
        })
        
        return full_response
    
    def get_stats(self):
        """Get usage statistics"""
        return {
            "total_tokens": self.total_tokens,
            "messages": len([m for m in self.conversation_history if m['role'] == 'user'])
        }

print("✅ SimpleChatbot class created!")

### Test the Chatbot

In [ ]:
# Initialize chatbot
bot = SimpleChatbot(api_key=os.environ.get('OPENAI_API_KEY'))

# Set personality
bot.set_system_message(
    "You are a helpful and friendly AI assistant. "
    "Keep responses concise (under 100 words)."
)

# Have a conversation
print("Chatbot: Hello! How can I help you today?\n")

# Message 1
print("You: What is Python?")
print("Chatbot: ", end="")
bot.chat("What is Python?", stream=True)
print()

# Message 2
print("You: What can I use it for?")
print("Chatbot: ", end="")
bot.chat("What can I use it for?", stream=True)
print()

# Show stats
print(f"\nStats: {bot.get_stats()}")

## Part 7: Capstone - SupportGenie v0.1

Let's build the first version of our capstone project!

In [ ]:
class SupportGenieV1(SimpleChatbot):
    """
    SupportGenie - AI Customer Support Assistant
    Version 0.1: Basic chatbot with professional tone
    """
    
    def __init__(self, api_key):
        super().__init__(api_key, model="gpt-3.5-turbo")
        
        # Set professional customer service personality
        self.set_system_message("""
You are SupportGenie, an AI customer support assistant for TechStore.

Guidelines:
- Be professional, empathetic, and helpful
- Keep responses under 100 words
- If you don't know something, be honest
- Always offer to escalate to a human agent if needed
- Use a friendly but professional tone

Response Format:
1. Acknowledge the customer's concern
2. Provide helpful information or solution
3. Ask if there's anything else you can help with
        """)
    
    def welcome(self):
        """Display welcome message"""
        print("="* 60)
        print("    SupportGenie v0.1 - AI Customer Support")
        print("="* 60)
        print("\nHello! I'm SupportGenie, your AI support assistant.")
        print("How can I help you today?\n")

print("✅ SupportGenie v0.1 created!")

### Test SupportGenie

In [ ]:
# Initialize SupportGenie
support_bot = SupportGenieV1(api_key=os.environ.get('OPENAI_API_KEY'))
support_bot.welcome()

# Test conversation
test_queries = [
    "My order hasn't arrived yet",
    "I need to return a product",
    "What are your business hours?"
]

for query in test_queries:
    print(f"Customer: {query}")
    print("SupportGenie: ", end="")
    support_bot.chat(query, stream=True)
    print()

print(f"\nSession Stats: {support_bot.get_stats()}")

## 🎯 Final Challenge: Enhance SupportGenie

Add new features to SupportGenie:
1. Conversation history limit (keep last 10 messages)
2. Cost tracking
3. Response time measurement

In [ ]:
# Your enhanced version here
class SupportGenieV2(SupportGenieV1):
    # Add your enhancements
    pass

## Session 1 Complete! ✅

### What You Learned:
- ✅ How LLMs work (token prediction)
- ✅ Token counting and cost calculation
- ✅ Making API calls to OpenAI
- ✅ Key parameters (temperature, max_tokens)
- ✅ Streaming responses
- ✅ Building a chatbot
- ✅ SupportGenie v0.1

### Next Session:
**Session 2: Prompt Engineering** - Learn to write effective prompts to make SupportGenie smarter!

### Homework:
1. Complete the enhancement challenge
2. Experiment with different temperatures
3. Try other OpenAI models (gpt-4)
4. Calculate costs for 100 conversations

---

**Great job!** 🎉